In [ ]:
%%bash
pip install instructure-dap-client

In [ ]:
from datetime import datetime
import subprocess
import multiprocessing

def run_bash_command(command):
    subprocess.run(command, shell=True)

api_url = "https://api-gateway.instructure.com"
client_id = "us-west-2#8cb98924-70a9-49b1-bfe9-393e6ae111d5"
client_secret = "3zlv-vTSxyZvnR7grsv56PYBnAgy_N4qug1DP-YIzQs"

# define tables to be extracted from Canvas
tables = [
    "users"
]

commands = []

# create string timestamp to serve as directory
now = datetime.now()
date_ts = now.strftime("%Y%m%d%H%M%S")

# build bash command for the incremental load of each table
for table in tables:

    # retrieve most-recent load timestamp for 'since' parameter
    query  = f"SELECT MAX(ts) FROM {table}"
    result = spark.sql(query).collect()
    #since  = result[0][0]
    since = "2023-05-03T20:24:58"

    # build the command string
    file_path = f"/lakehouse/default/Files/Canvas/{table}/{date_ts}"
    commands.append(f"dap --base-url {api_url} --client-id {client_id} --client-secret {client_secret} incremental --format parquet --output-directory {file_path} --table {table} --namespace canvas --since {since}")


# create and start a separate process for each command
processes = [multiprocessing.Process(target=run_bash_command, args=(cmd,)) for cmd in commands]

for process in processes:
    process.start()

# wait for all processes to complete
for process in processes:
    process.join()

In [ ]:
# perform a merge between each staging table and delta table
for table in tables:

    # transform the incremental data and store to a temporary table
    df = spark.read.parquet(f"Files/Canvas/{table}/{date_ts}/*/*.parquet")
    flat_df = df.select("key.*", "value.*", "meta.*")
    flat_df.createOrReplaceTempView(f"{table}_stage")

    updates = []
    inserts = []

    # dynamically build sql query components 
    for value in df.select("value.*").columns:
        
        updates.append(f"t.{value} = s.{value}")
        inserts.append(value)

    update  = "\n\t\t\t, ".join(updates)
    insert  = "\n\t\t\t, ".join(inserts)
    insert2 = "\n\t\t\t, s.".join(inserts)

    sql = f"""
        MERGE INTO {table} t
        USING {table}_stage s
        ON t.id = s.id
        WHEN MATCHED AND s.action = 'U' THEN 
        UPDATE SET
            {update}
            , t.ts = s.ts
        WHEN MATCHED AND s.action = 'D' THEN
        DELETE 
        WHEN NOT MATCHED AND s.action = 'I' THEN
        INSERT (
            id
            , {insert}
            , ts
        ) VALUES (
            s.id
            , {insert2}
            , s.ts
        )
    """

    # execute merge and show results
    display(spark.sql(sql))